In [21]:
import numpy as np
import pandas as pd
import pyarrow.feather as feather
import plotly.express as px
import warnings

# Filter out FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [22]:
df = feather.read_table(r"C:\Users\mexic\Documents\Python\data\cumulative_2006-2022.feather").to_pandas()

#df.head()

After importing data, we will focus on tracking changes on a state to state level across all years in the sample. First adding a State Abbreviation column which will be needed later to visualize the data on a map.

In [23]:
#add state abbbv column, ie FL CA etc
fips_to_state = {
    1: 'AL', 2: 'AK', 4: 'AZ', 5: 'AR', 6: 'CA', 8: 'CO', 9: 'CT', 10: 'DE',
    11: 'DC', 12: 'FL', 13: 'GA', 15: 'HI', 16: 'ID', 17: 'IL', 18: 'IN',
    19: 'IA', 20: 'KS', 21: 'KY', 22: 'LA', 23: 'ME', 24: 'MD', 25: 'MA',
    26: 'MI', 27: 'MN', 28: 'MS', 29: 'MO', 30: 'MT', 31: 'NE', 32: 'NV',
    33: 'NH', 34: 'NJ', 35: 'NM', 36: 'NY', 37: 'NC', 38: 'ND', 39: 'OH',
    40: 'OK', 41: 'OR', 42: 'PA', 44: 'RI', 45: 'SC', 46: 'SD', 47: 'TN',
    48: 'TX', 49: 'UT', 50: 'VT', 51: 'VA', 53: 'WA', 54: 'WV', 55: 'WI',
    56: 'WY'
}

df['state_abbv'] = df['state'].map(fips_to_state)

Next we will show a simple example of how to calculate the proportion of a simple column, Sex. Note, we are not accounting for the survey weights here.

In [24]:
#Example calculating raw Proportion of Male/Females in the sample

# remove NAs
df_sex = df[df['sex'].notna()].copy()

#convert to categorical Male/Female
df_sex['sex2'] = df_sex['sex'].replace({1.0: "Male",
                                        2.0: "Female"})


# Group by 'Year', 'State_abbv', and 'Sex' and count the number of each sex in each group
count_table = df_sex.groupby(['year', 'state_abbv', 'sex2']).size().reset_index(name='Count')

# Pivot the table to have 'Year' and 'State_abbv' as the index, 'Sex' as columns, and 'Count' as values
pivot_table = count_table.pivot_table(index=['year', 'state_abbv'], columns='sex2', values='Count', fill_value=0)

# Calculate the proportion of Male and Female
pivot_table['Male Proportion'] = pivot_table['Male'] / (pivot_table['Male'] + pivot_table['Female']) * 100
pivot_table['Female Proportion'] = pivot_table['Female'] / (pivot_table['Male'] + pivot_table['Female']) * 100

# Reset the index to have 'Year' and 'State_abbv' as columns
pivot_table.reset_index(inplace=True)

# Display the resulting table
print(pivot_table)

sex2  year state_abbv  Female    Male  Male Proportion  Female Proportion
0     2006         AK    75.0    56.0        42.748092          57.251908
1     2006         AL   311.0   184.0        37.171717          62.828283
2     2006         AR   199.0   157.0        44.101124          55.898876
3     2006         AZ   471.0   492.0        51.090343          48.909657
4     2006         CA  1801.0  1836.0        50.481166          49.518834
..     ...        ...     ...     ...              ...                ...
760   2020         VT    85.0    65.0        43.333333          56.666667
761   2020         WA   728.0   646.0        47.016012          52.983988
762   2020         WI   716.0   518.0        41.977310          58.022690
763   2020         WV   261.0   183.0        41.216216          58.783784
764   2020         WY    55.0    40.0        42.105263          57.894737

[765 rows x 6 columns]


Finally, we will create a pivot table with a column with more than two categories, ideo5, and account for the survey weights across all years. The CCES makes survey weights comparible accross years, with a column called weight_cumulative. 

In [50]:
## Create similar poportion table for ideology, ideo5, which is self reported ideology 

# filter out NAs from ideology respondants
df_ideo = df[df['ideo5'].notna()].copy()
#print(df_ideo['ideo5'].unique)

# apply survey weights to the ideo5 column before finding weighted sum
# Group by 'Year', 'State_abbv', and 'Sex' and count the number of each sex in each group
count_table = df_ideo.groupby(['year', 'state_abbv', 'ideo5'])['weight_cumulative'].sum().reset_index(name='Count')

# Pivot the table to have 'Year' and 'State_abbv' as the index, 'Sex' as columns, and 'Count' as values
pivot_table = count_table.pivot_table(index=['year', 'state_abbv'], columns='ideo5', values='Count', fill_value=0)
#print(pivot_table)

# Calculate the proportion of ideologies
categories = ['Very Liberal', 'Liberal', 'Moderate', 'Conservative', 'Very Conservative', 'Not Sure']  # List of all possible categories
for category in categories:
    pivot_table[f'{category} Proportion'] = pivot_table[category] / pivot_table[categories].sum(axis=1) * 100


# Reset the index to have 'Year' and 'State_abbv' as columns
pivot_table.reset_index(inplace=True)

# see what resulting columns look like for a given year and state
fl_2012_prop = pivot_table[(pivot_table['year'] == 2012) & (pivot_table['state_abbv'] == 'FL')]
print(fl_2012_prop)

# actually add the columns
print(fl_2012_prop.iloc[:,-6:].sum(axis = 1))

315    100.0
dtype: float64


Now we move on to creating a visualization for the variable of interest, self-reported ideology, across the country and for all years of the sample. There is a sample test_plot with a smaller dataset in the test_plots file.

In [53]:
# Create an animated choropleth map using Plotly
fig = px.choropleth(
    pivot_table,
    locations="state_abbv",
    locationmode="USA-states",
    color="Conservative Proportion",
    animation_frame="year",  # Specify the column for animation
    scope="usa",
    color_continuous_scale="Reds",
    title="Choropleth Map Over Years",
    range_color=(pivot_table['Very Conservative Proportion'].min(), pivot_table['Very Conservative Proportion'].max())  # Set the color scale range
)

fig.update_geos(
    resolution=50,
    showcoastlines=True,
    coastlinecolor="Black",
    showland=True,
    landcolor="white"
)

fig.show()